In [1]:
from transformers import RagTokenizer, RagRetriever, RagSequenceForGeneration
from datasets import load_dataset
import torch

In [2]:
def load_data(dataset_name):
    data_files = {"train": "train.json", "validation": "validation.json", "test": "test.json"}
    data = load_dataset(dataset_name, data_files=data_files)
    return data

def get_data_subset(data, size=10):
    examples = []
    for i in range(0, size):
        examples.append(data[i]['title'] + "\n" + data[i]['selftext'])
    return examples

def initialize(model_name):
    tokenizer = RagTokenizer.from_pretrained(model_name) 
    retriever = RagRetriever.from_pretrained(model_name, index_name="compressed", use_dummy_dataset=False) # use dummy for now
    model = RagSequenceForGeneration.from_pretrained(model_name, retriever=retriever) 
    model = model.to(device)
    return (tokenizer, retriever, model)

def infer(tokenizer, retriever, model, inputs):
    input_dict = tokenizer.prepare_seq2seq_batch(inputs, return_tensors="pt").to(model.device)
    generated = model.generate(input_ids=input_dict["input_ids"]) 
    output = tokenizer.batch_decode(generated, skip_special_tokens=True)
    return output

In [3]:
eli5 = load_data("vblagoje/lfqa")
eli5_train = eli5['train']
subset = get_data_subset(eli5_train, size=20)
device = "cuda:0" if torch.cuda.is_available() else "cpu"

Found cached dataset json (/work/pi_adrozdov_umass_edu/gdhanania_umass_edu/hf_cache/datasets/vblagoje___json/vblagoje--lfqa-81a0582aaded411a/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e)


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
(tokenizer, retriever, model) = initialize("facebook/rag-sequence-nq")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizerFast'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called fr

In [5]:
output = infer(tokenizer, retriever, model, subset)
print(output)

/home/gdhanania_umass_edu/.conda/envs/cont-base/lib/python3.10/site-packages/transformers/models/rag/tokenization_rag.py:92: FutureWarning: `prepare_seq2seq_batch` is deprecated and will be removed in version 5 of 🤗 Transformers. Use the regular `__call__` method to prepare your inputs and the tokenizer under the `with_target_tokenizer` context manager to prepare your targets. See the documentation of your specific tokenizer for more details
  warnings.warn(
2023-04-28 02:40:46.199476: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-28 02:40:49.406077: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


['', ' are there any other subreddits', ' when a body part hurts', ' fugitive slaves', ' interregnum', ' disbanded', ' india', ' cat', ' 25,000 genes', ' are not hispanics', ' a concussion', ' dispatcher', '', ' slowly', ' activation', ' prevents it from exploding', ' the dow', ' 1962', '', ' pia mater']
